In [1]:
import pymongo
import numpy as np

db = pymongo.MongoClient()['mec']
dataset = 'calzedoniajson_items_0'

df, tf = {}, {}
docs = {}
cursor = db[dataset].find(no_cursor_timeout=True).sort('_id', pymongo.ASCENDING)
size = cursor.count()
for i, doc in enumerate(cursor):
    docs[i] = {'d': doc['_id'], 'tags': []}
    try:
        tags = doc['clarifai_tags']
        probs = doc['clarifai_probs']
        docs[i]['tags'] = tags
        for j, tag in enumerate(tags):
            try:
                posting = tf[tag]
            except KeyError:
                posting = {}
            posting[i] = {'raw_tf': 1, 'prob': probs[j], 
                            'norm_p': probs[j] / max(probs),
                           'freq': 1.0 / len(tags)}
            tf[tag] = posting
            try:
                df[tag]['df'] += 1
            except KeyError:
                df[tag] = {'df': 1.0}
            df[tag]['idf'] = np.log(1 + float(size) / df[tag]['df'])  
    except KeyError:
        pass
cursor.close()


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

In [ ]:
def get_tf(tag, doc, metric='norm_p'):
    try:
        w = tf[tag][doc][metric]
    except KeyError:
        w = 0.0
    return w

def get_idf(tag, metric='idf'):
    try:
        w = df[tag][metric]
    except KeyError:
        w = 0.0
    return w

def bag(doc, tfm='norm_p', idfm='idf'):
    try:
        tgs = docs[doc]['tags']
        b = np.array([get_tf(x, doc, metric=tfm) * get_idf(x, metric=idfm)
                     for x in tgs])
    except KeyError:
        tgs = []
        b = np.array([])
    return tgs, b

def get_posting(tag, tfm='norm_p', idfm='idf'):
    p = []
    idf = get_idf(tag, metric=idfm)
    try:
        pd = tf[tag]
        for d, m in pd.items():
            p.append((d, m[tfm]*idf))
    except KeyError:
        pass
    return sorted(p, key=lambda x: -x[0])

# print get_posting('girl')
    

In [ ]:
tag_list = tf.keys()
tag_len, tag_avg_w, tag_idf = [], [], []
for t in tag_list:
    p = get_posting(t)
    tag_len.append(len(p))
    tag_avg_w.append(np.mean([x[1] for x in p]))
    tag_idf.append(get_idf(t))

idf_y, w_y, l_y = [], [], []
for i, x in sorted(enumerate(tag_idf), key=lambda x: -x[1]):
    idf_y.append(x)
    w_y.append(tag_avg_w[i])
    l_y.append(max(tag_len[i:]))

import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
axes[0].plot(idf_y)
axes[1].scatter(idf_y, w_y, alpha=0.2)
plt.show()


In [ ]:
def eucl_norm(lst):
    return np.sqrt(np.sum(map(lambda z: z**2, lst)))

def cosine(d1, d2):
    b1, w1 = bag(d1)
    b2, w2 = bag(d2)
    
    common_tags = [d for d in b1 if d in b2]
    m1 = np.array(map(lambda t: w1[b1.index(t)], common_tags))
    m2 = np.array(map(lambda t: w2[b2.index(t)], common_tags))
    e = eucl_norm(w1) * eucl_norm(w2)
    if e > 0:
        return np.sum((m1 * m2)/e)
    else:
        return 0.0

print cosine(0,1) 

In [ ]:
def naive_matching(d1):
    mappings = []
    for d in docs.keys():
        if d != 1:
            mappings.append((d, cosine(d1, d)))
    return sorted(mappings, key=lambda x: -x[1])

# Inefficienze:
# scorro tutti i documenti, anche quelli senza tag in comune
# ricalcolo la bag of words di tanti elementi che non mi serviranno mai

In [ ]:
def almost_matching(d1):
    result = {}
    score = {}
    tags, w = bag(d1)
    for j, t in enumerate(tags):
        d_w = w[j]
        for doc_id, v in get_posting(t):
            if doc_id != d1:
                try:
                    result[doc_id]
                except KeyError:
                    result[doc_id] = []
                result[doc_id].append((v, d_w))
    
    for doc_id, lst in result.items():
        m1 = np.array(map(lambda x: x[0], lst))
        m2 = np.array(map(lambda x: x[1], lst))
        score[doc_id] = np.sum((m1 * m2)/(eucl_norm(m1)*eucl_norm(m2)))
    return sorted(score.items(), key=lambda x: -x[1])

# In questo caso la cosine similarity è calcolata in modo sbagliato perché eucl_norm è applicato solo ai pesi dei tag comuni e non a tutti a quelli associati al documento
# Posso aggiungermi nella posting il n. di tag di ciascun documento oppure posso salvarmi a livello di documento la sua eucl_norm


In [ ]:
# Verifichiamo la similutidine tra i tag
# Costruiamo una bag of words con spazio tag e vettore documento

def tag_bag(tag, tfm='norm_p', idfm='idf'):
    dcs, prbs = [], []
    idf = get_idf(tag, metric=idfm)
    try:
        pd = tf[tag]
        for d, m in pd.items():
            dcs.append(d)
            prbs.append(m[tfm]*idf)
    except KeyError:
        pass
    return dcs, prbs

print tag_bag('fashion')

In [ ]:
def tag_cosine(t1, t2):
    p1 = dict(get_posting(t1))
    p2 = dict(get_posting(t2))
    L1 = sum([np.power(x,2) for x in p1.values()])
    L2 = sum([np.power(x,2) for x in p2.values()])
    e = np.sqrt(L1)*np.sqrt(L2)
    p = 0.0
    if e > 0:
        for t1, w1 in p1.items():
            try:
                w2 = p2[t1]
                p += w1*w2
            except KeyError:
                pass
        return p/e
    else:
        return 0.0

print tag_cosine('woman', 'beach')

def naive_tag_matching(t1):
    mappings = []
    for t2 in df.keys():
        if t1 != t2:
            mappings.append((t2, tag_cosine(t1,t2)))
    return sorted(mappings, key= lambda x: -x[1])

In [ ]:
tag_map = naive_tag_matching('wet')
for m in tag_map:
    print m[0], m[1]